In [1]:
from scripts import support, metrics, vectorization as v, aggregation
from scripts.kuroda import kuroda
from scripts.insd import insd
from scripts.nras import nras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from scipy.sparse import csr_matrix
%matplotlib inline

In [2]:
#CODES


df = pd.read_excel("./data/Supply_2007_2012_DET.xlsx", "NAICS Codes", skiprows=4)
df = df.iloc[:-6,:-1]
df = df.dropna(axis=0, how = 'all')
df = df.fillna(0)

codes = np.array(df)

In [3]:
#DET 2007


df = pd.read_excel("./data/Supply_2007_2012_DET.xlsx", "2007")
df = df.iloc[5:-3,2:-12]
df = df.fillna(0)

DET_2007 = np.array(df)
DET_2007 = np.where(DET_2007=='...', 0, DET_2007)
dt_2007 = v.tovector(DET_2007).astype(float)
# np.shape(DET_2007)

In [4]:
#DET 2012


df = pd.read_excel("./data/Supply_2007_2012_DET.xlsx", "2012")
df = df.iloc[5:-3,2:-12]
df = df.fillna(0)

DET_2012 = np.array(df)
DET_2012 = np.where(DET_2012=='...', 0, DET_2012)
dt_2012 = v.tovector(DET_2012).astype(float)


#Зануляем элементы, в которых меняется знак
det_2007 = dt_2007
det_2012 = dt_2012
det_2007[dt_2007 * dt_2012 < 0] = 0
det_2012[dt_2007 * dt_2012 < 0] = 0
DET_2012[DET_2007 * DET_2012 < 0] = 0


#Считаем итоги по строкам и столбцам
rows_sums_2012 = np.sum(DET_2012, axis=1)
columns_sums_2012 = np.sum(DET_2012, axis=0)
totals_2012 = np.concatenate((rows_sums_2012, columns_sums_2012)).astype(float)[np.newaxis].T
# DET_2012

In [5]:
#SUM 2007


df = pd.read_excel("./data/Supply_1997-2017_SUM.xlsx", "2007")
df = df.iloc[6:-1,2:-12]
df = df.fillna(0)

SUM_2007 = np.array(df)
SUM_2007 = np.where(SUM_2007=='...', 0, SUM_2007)

SUM_2007[[72, 71]] = SUM_2007[[71, 72]]
sum_2007 = v.tovector(SUM_2007).astype(float)
# np.shape(SUM_2007)

In [6]:
#SUM 2012


df = pd.read_excel("./data/Supply_1997-2017_SUM.xlsx", "2012")
df = df.iloc[6:-1,2:-12]
df = df.fillna(0)

SUM_2012 = np.array(df)
SUM_2012 = np.where(SUM_2012=='...', 0, SUM_2012)

SUM_2012[[72, 71]] = SUM_2012[[71, 72]]
sum_2012 = v.tovector(SUM_2012).astype(float)
# SUM_2012

In [7]:
#SECT 2007


df = pd.read_excel("./data/Supply_1997-2017_SECT.xlsx", "2007")
df = df.iloc[6:-1, 2:-11]
df = df.fillna(0)

SECT_2007 = np.array(df)
SECT_2007 = np.where(SECT_2007=='...', 0, SECT_2007)
sect_2007 = v.tovector(SECT_2007).astype(float)
# SECT_2007

In [8]:
#SECT 2012


df = pd.read_excel("./data/Supply_1997-2017_SECT.xlsx", "2012")
df = df.iloc[6:-1, 2:-11]
df = df.fillna(0)

SECT_2012 = np.array(df)
SECT_2012 = np.where(SECT_2012=='...', 0, SECT_2012)
sect_2012 = v.tovector(SECT_2012).astype(float)
# SECT_2012

In [9]:
#parsing aggregation codes


sect_rows, sect_columns, sum_rows, sum_columns = aggregation.parse_agg(codes)

In [10]:
#Составляем матрицы-ключи агрегации и векторизованный вид
#Для аггрегации в SUM (73,71)

left_sum = aggregation.make_keym(405, 73, sum_rows)
right_sum = aggregation.make_keym(405, 71, sum_columns)

g_to_sum = csr_matrix(aggregation.keys_to_g(left_sum, right_sum))

In [11]:
#Составляем матрицы-ключи агрегации и векторизованный вид
#Для аггрегации в SECT (17,15)

left_sect = aggregation.make_keym(405, 17, sect_rows)
right_sect = aggregation.make_keym(405, 15, sect_columns)

g_to_sect = csr_matrix(aggregation.keys_to_g(left_sect, right_sect))

In [12]:
true_sum_2012 = g_to_sum @ det_2012

In [13]:
true_sect_2012 = g_to_sect @ det_2012

In [14]:
from scripts import support, metrics, vectorization as v, aggregation
def experiment(tables_kind, method_name, bench_matrix, origin_matrix, target_matrix, experiment_type, sparsed = True, save = True):
    """
    Получение результата эксперимента

    Parameters
    ----------
    method_name: string
        метод для получения новой матрицы
    bench_matrix: np.array или sparse матрица
        бенчмарк таблица
    origin_matrix: np.array или sparse матрица
        оригинальная таблица
    target_matrix: np.array или sparse матрица
        целевая таблица
    experiment_type: string
        агрегация\дезаггрегация\нет
    sparse: bool
        использовать sparse матрицу или np.array
    Returns
    -------
    a: float
        значение метрики
    """
    if experiment_type == 'дезаггрегация':
        if tables_kind == 'sum':
            if sparsed:
                g = g_to_sum
            else:
                g = aggregation.keys_to_g(left_sum, right_sum)
        else:
            if sparsed:
                g = g_to_sect
            else:
                g = aggregation.keys_to_g(left_sect, right_sect)
    
    # For Kuroda
    if len(method_name.split('_')) > 1:
        
        method, mtype = method_name.split('_')
        method = eval(method)
        results = np.round(method(g, eval(bench_matrix), eval(origin_matrix), mtype=int(mtype), sparsed=sparsed))
    else:
        method = eval(method_name)
        results = method(g, eval(bench_matrix), eval(origin_matrix), sparsed=sparsed)
    metrics_values = metrics.get_values(np.round(results), eval(target_matrix), save=True)
    
    # Save metrics' results
    if save:
        info = {'method':method_name, 'experiment_type':experiment_type, 'target_matrix':target_matrix, 
                'origin_matrix':origin_matrix, 'bench_matrix':bench_matrix}
        info = {**info,**metrics_values}

        res_table = pd.read_excel('results/results_supply.xlsx')
        res_table = res_table.append([info])
        res_table.to_excel('results/results_supply.xlsx', index=None,encoding='utf-8')

    return metrics_values 


In [15]:
experiment('sum', 'nras', 'det_2007', 'true_sum_2012', 'det_2012', 'дезаггрегация', True)

1


{'N0': 1139,
 'PSISTAT': 0.011,
 'RSQ': 0.9905,
 'MAPE': 7.6132,
 'SWAD': 0.0511,
 'WAPE': 11.2657}

In [16]:
experiment('sum', 'insd', 'det_2007', 'true_sum_2012', 'det_2012', 'дезаггрегация', True)

Academic license - for non-commercial use only
Parameter BarConvTol unchanged
   Value: 1e-08  Min: 0.0  Max: 1.0  Default: 1e-08
Parameter BarQCPConvTol unchanged
   Value: 1e-06  Min: 0.0  Max: 1.0  Default: 1e-06
Changed value of parameter DualReductions to 0
   Prev: 1  Min: 0  Max: 1  Default: 1


{'N0': 981,
 'PSISTAT': 0.011,
 'RSQ': 0.9905,
 'MAPE': 7.7892,
 'SWAD': 0.0511,
 'WAPE': 11.2658}

In [17]:
experiment('sum', 'kuroda_3', 'det_2007', 'true_sum_2012', 'det_2012', 'дезаггрегация', sparsed = True)

Parameter BarConvTol unchanged
   Value: 1e-08  Min: 0.0  Max: 1.0  Default: 1e-08
Parameter BarQCPConvTol unchanged
   Value: 1e-06  Min: 0.0  Max: 1.0  Default: 1e-06
Parameter DualReductions unchanged
   Value: 1  Min: 0  Max: 1  Default: 1


{'N0': 883,
 'PSISTAT': 0.011,
 'RSQ': 0.9905,
 'MAPE': 7.8882,
 'SWAD': 0.0511,
 'WAPE': 11.2639}

In [18]:
experiment('sum', 'kuroda_2', 'det_2007', 'true_sum_2012', 'det_2012', 'дезаггрегация', sparsed = False)

Parameter BarConvTol unchanged
   Value: 1e-08  Min: 0.0  Max: 1.0  Default: 1e-08
Parameter BarQCPConvTol unchanged
   Value: 1e-06  Min: 0.0  Max: 1.0  Default: 1e-06
Parameter DualReductions unchanged
   Value: 1  Min: 0  Max: 1  Default: 1


{'N0': 772,
 'PSISTAT': 0.011,
 'RSQ': 0.9905,
 'MAPE': 7.7889,
 'SWAD': 0.0511,
 'WAPE': 11.2584}

In [19]:
experiment('sum', 'kuroda_1', 'det_2007', 'true_sum_2012', 'det_2012', 'дезаггрегация', sparsed = False)

Parameter BarConvTol unchanged
   Value: 1e-08  Min: 0.0  Max: 1.0  Default: 1e-08
Parameter BarQCPConvTol unchanged
   Value: 1e-06  Min: 0.0  Max: 1.0  Default: 1e-06
Parameter DualReductions unchanged
   Value: 1  Min: 0  Max: 1  Default: 1


{'N0': 770,
 'PSISTAT': 0.011,
 'RSQ': 0.9905,
 'MAPE': 7.7895,
 'SWAD': 0.0511,
 'WAPE': 11.2582}

In [20]:
experiment('sect', 'nras', 'det_2007', 'true_sect_2012', 'det_2012', 'дезаггрегация', True)

1


{'N0': 834,
 'PSISTAT': 0.0152,
 'RSQ': 0.9845,
 'MAPE': 9.4783,
 'SWAD': 0.0864,
 'WAPE': 14.9561}

In [21]:
experiment('sect', 'insd', 'det_2007', 'true_sect_2012', 'det_2012', 'дезаггрегация', True)

Parameter BarConvTol unchanged
   Value: 1e-08  Min: 0.0  Max: 1.0  Default: 1e-08
Parameter BarQCPConvTol unchanged
   Value: 1e-06  Min: 0.0  Max: 1.0  Default: 1e-06
Changed value of parameter DualReductions to 0
   Prev: 1  Min: 0  Max: 1  Default: 1


{'N0': 834,
 'PSISTAT': 0.0152,
 'RSQ': 0.9845,
 'MAPE': 9.4799,
 'SWAD': 0.0864,
 'WAPE': 14.9565}

In [22]:
experiment('sect', 'kuroda_3', 'det_2007', 'true_sect_2012', 'det_2012', 'дезаггрегация', sparsed = True)

Parameter BarConvTol unchanged
   Value: 1e-08  Min: 0.0  Max: 1.0  Default: 1e-08
Parameter BarQCPConvTol unchanged
   Value: 1e-06  Min: 0.0  Max: 1.0  Default: 1e-06
Parameter DualReductions unchanged
   Value: 1  Min: 0  Max: 1  Default: 1


{'N0': 1888,
 'PSISTAT': 0.0171,
 'RSQ': 0.984,
 'MAPE': 5.7522,
 'SWAD': 0.0864,
 'WAPE': 15.1468}

In [23]:
experiment('sect', 'kuroda_2', 'det_2007', 'true_sect_2012', 'det_2012', 'дезаггрегация', sparsed = False)

Parameter BarConvTol unchanged
   Value: 1e-08  Min: 0.0  Max: 1.0  Default: 1e-08
Parameter BarQCPConvTol unchanged
   Value: 1e-06  Min: 0.0  Max: 1.0  Default: 1e-06
Parameter DualReductions unchanged
   Value: 1  Min: 0  Max: 1  Default: 1


{'N0': 834,
 'PSISTAT': 0.0152,
 'RSQ': 0.9845,
 'MAPE': 9.4796,
 'SWAD': 0.0864,
 'WAPE': 14.9568}

In [24]:
experiment('sect', 'kuroda_1', 'det_2007', 'true_sect_2012', 'det_2012', 'дезаггрегация', sparsed = False)

Parameter BarConvTol unchanged
   Value: 1e-08  Min: 0.0  Max: 1.0  Default: 1e-08
Parameter BarQCPConvTol unchanged
   Value: 1e-06  Min: 0.0  Max: 1.0  Default: 1e-06
Parameter DualReductions unchanged
   Value: 1  Min: 0  Max: 1  Default: 1


{'N0': 834,
 'PSISTAT': 0.0152,
 'RSQ': 0.9845,
 'MAPE': 9.4796,
 'SWAD': 0.0864,
 'WAPE': 14.9568}